In [1]:
project_name="mice"

In [2]:
import os

In [3]:
%pwd

'f:\\ML_Ops_learning\\Mice-Protein-Expression\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'f:\\ML_Ops_learning\\Mice-Protein-Expression'

MLFLOW_TRACKING_URI=https://dagshub.com/SunilKumar-ugra/Mice-Protein-Expression.mlflow \
MLFLOW_TRACKING_USERNAME=SunilKumar-ugra \
MLFLOW_TRACKING_PASSWORD=ff039c8b90d721cb1c7f99b788c8bfe02f4d345b \
python script.py

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/SunilKumar-ugra/Mice-Protein-Expression.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="SunilKumar-ugra"
os.environ["MLFLOW_TRACKING_PASSWORD"]="ff039c8b90d721cb1c7f99b788c8bfe02f4d345b"

## config/config.yaml

In [ ]:
%%writefile -a config/config.yaml

model_evaluation:
  root_dir: artifacts/model_evaluation
  test_data_path: artifacts/data_transformation/test.csv
  model_path: artifacts/model_trainer/model.joblib
  metric_file_name: artifacts/model_evaluation/metrics.json

# Entity Folder
1. Goto entity folder  src-> wine_quality -> entity-> __init__.py 
2. Paste below code in __init__.py constructor file

In [7]:
project_name 

'mice'

In [8]:
from dataclasses import dataclass
from pathlib import Path

In [10]:
#%%writefile -a src/{project_name}/entity/__init__.py


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


# Config folder

1. Copy entire ConfigurationManager class
2. Goto config folder src->wine_quality->config->configuration.py and paste entire ConfigurationManager class in configuration.py
3. Import DataIngestionConfig from wine_quality.entity using this line  
from wine_quality.entity import DataIngestionConfig

In [11]:
project_name

'mice'

In [12]:
from mice.constants import *
from mice.utils import read_yaml, create_directories, save_json

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/SunilKumar-ugra/Mice-Protein-Expression.mlflow",
           
        )

        return model_evaluation_config


In [27]:
# %%writefile -a src/{project_name}/config/__init__.py   

    
#     def get_model_evaluation_config(self) -> ModelEvaluationConfig:
#         config = self.config.model_evaluation
#         params = self.params.ElasticNet
#         schema =  self.schema.TARGET_COLUMN

#         create_directories([config.root_dir])

#         model_evaluation_config = ModelEvaluationConfig(
#             root_dir=config.root_dir,
#             test_data_path=config.test_data_path,
#             model_path = config.model_path,
#             all_params=params,
#             metric_file_name = config.metric_file_name,
#             target_column = schema.name,
#             mlflow_uri="https://dagshub.com/SunilKumar-ugra/Mice-Protein-Expression.mlflow",
           
#         )

#         return model_evaluation_config

Appending to src/mice/config/__init__.py


# Component folder

1. Copy entire DataIngestion class
2. Goto Component folder [ src-> wine_quality -> components - > data_ingestion.py ]
2. Paste DataIngestionclass in data_ingestion.py.py
3. Import dataIngestionconfig from entity using this line 

from projectname.entity import dataIngestionconfig

In [16]:
project_name 

'mice'

In [18]:
# %%writefile -a src/{project_name}/components/model_evaluation.py
# from mice.entity import ModelEvaluationConfig
# from mice.utils import save_json

In [20]:
# %%writefile -a src/{project_name}/components/model_evaluation.py
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from pathlib import Path

In [22]:
# %%writefile -a src/{project_name}/components/model_evaluation.py

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column,'MouseID'], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [23]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-11-30 22:34:47,975] - mice - INFO - __init__ file - LineNum:32   - yaml file: config\config.yaml loaded successfully
[2023-11-30 22:34:47,980] - mice - INFO - __init__ file - LineNum:32   - yaml file: params.yaml loaded successfully
[2023-11-30 22:34:48,014] - mice - INFO - __init__ file - LineNum:32   - yaml file: schema.yaml loaded successfully
[2023-11-30 22:34:48,018] - mice - INFO - __init__ file - LineNum:52   - created directory at: artifacts
[2023-11-30 22:34:48,021] - mice - INFO - __init__ file - LineNum:52   - created directory at: artifacts/model_evaluation
[2023-11-30 22:34:50,019] - mice - INFO - __init__ file - LineNum:66   - json file saved at: artifacts\model_evaluation\metrics.json


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/11/30 22:35:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.


In [24]:
#%%writefile src/{project_name}/pipeline/stage_05_model_evaluation.py
from mice.config import ConfigurationManager
from mice.components.model_evaluation import ModelEvaluation

class EvaluationPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        eval_config = config.get_evaluation_config()
        evaluation = ModelEvaluation(eval_config)
        evaluation.log_into_mlflow()

Overwriting src/mice/pipeline/stage_05_model_evaluation.py


In [25]:
%%writefile -a main.py


from mice.pipeline.stage_05_model_evaluation import EvaluationPipeline

STAGE_NAME = "Evaluation stage"
try:
   logger.info(f"*******************")
   logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
   model_evalution = EvaluationPipeline()
   model_evalution.main()
   logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")

except Exception as e:
        logger.exception(e)
        raise e

Appending to main.py


In [29]:
os.system("python main.py")

0